We have to run this part for all instantiations.

In [8]:
import collections
import random
from math import *
import scipy.optimize as opt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if(n<=0):
        return 0.
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        return 0
    tot=0
    val=n
    for i in c:
        tot+=binomH(n,i)
        n-=i
    return tot


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

For Nested-2*

In [9]:
def reps1(n10,n00,e211,e201,e101,p): 
   #length of whole vec = p 
    if n10 <= 0.000001 or p == 0.:
        return 0
    if p < n10 or p - 2*n10 < 2*e101:
        return 0.
    
    return 2*multiH(n10, [e211,e211,n10*.5-e211]) + multiH(p-2*n10,[e201,e201,e101,e101])


def reps2(n11,n01,n21,e222,e212,e202,e102,p): 
    
    if n11 < 0.000001 or p == 0.:
        return 0.
    if p < n11 or p<n21 or p - 2*n11 - n21 < 2*e102:
        return 0.
    
    return 2*multiH(n21,[e222,e222])+2*multiH(n11, [e212, e212, n11*.5-e212])+multiH(n01, [e202, e202, e102, e102])



set_vars = collections.namedtuple('LWE', ' e211 e201 e101 e222 e212 e202 e102 g b')
def lwe(f) : return wrap(f, set_vars)


n10 = lambda x : x.b*x.g*w/2
n00 = lambda x : x.g*1 - 2*n10(x)

n21 = lambda x : x.e211+x.e201
n11 = lambda x : n10(x)/2+x.e101
n01 = lambda x : x.g*1-n10(x)-2*(x.e101+x.e211+x.e201)

n22 = lambda x : x.e222+x.e212+x.e202
n12 = lambda x : n11(x)/2+x.e102+n21(x)-2*x.e222
n02 = lambda x : x.g*1-n11(x)-2*n21(x)+2*(x.e222-x.e102-x.e212-x.e202)


def time(x):
    x=set_vars(*x)
    good_D = multiH(1,[w/2,w/2]) - ss(x)
    good_R = max(0,domain(x) - reps1(n10(x),n00(x),x.e211,x.e201,x.e101,x.g*1))
    collisions_when_R_good = domain(x) - 2*reps2(n11(x),n01(x),n21(x),x.e222,x.e212,x.e202,x.e102,x.g*1)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D




domain = lambda x : multiH(x.g*1,[n22(x), n22(x), n12(x), n12(x)])+ multiH((1-x.g)*.25,[(1-x.b*x.g)*w*.125,(1-x.b*x.g)*w*.125])
ss = lambda x : multiH(x.g,[x.g*x.b*w/2, x.g*x.b*w/2]) + 4*multiH((1-x.g)/4,[(1-x.g*x.b)*w*.125,(1-x.g*x.b)*w*.125])

constraints_lwe = [
# domain muss be equal to 1/2
    
{ 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)/2-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps1(n10(x),n00(x),x.e211,x.e201,x.e101,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps2(n11(x),n01(x),n21(x),x.e222,x.e212,x.e202,x.e102,x.g*1))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e211+n10(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e201+n00(x)/2)},
# { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e101+n00(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e101-x.e201+n00(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e222+n21(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e212+n11(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e202-x.e102+n01(x)/2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.b - (1/x.g)+(1/(x.g*w))-(1/w))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-x.b)}, 
{ 'type' : 'ineq',   'fun' : lwe(lambda x :  multiH(1,[w/2,w/2]) - ss(x))},

]

    
for i in range(25, 26):
    w = i*.02
    res = 100
    ress = 5000
    for j in range(5):
        start=[uniform(0,.009), uniform(0,.009), uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009),uniform(0,.009), uniform(.01,1), uniform(.01,1)]
        
        bounds=[(0.,0.1)]*7+[(.01,1)]*2
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
    print((round(w,4),round(res,4)))   

(0.5, 0.8425)


For Nested-1+

In [10]:
def reps(p, m, d, l): 
    """
    representations of length-l vector with p ones and m minus ones
    two length-l vectors with p/2+d ones and d minus ones each.
    """
    if p <= 0.000001 or l == 0.:
        return 0
    if l < p or l - p < 2*d:
        return 0.
    
    return p + m + multiH(l-p-m, [d,d]) 


set_vars = collections.namedtuple('LWE', 'd1 d2 g')
def lwe(f) : return wrap(f, set_vars)

def time(x):
    x=set_vars(*x)
    good_R = max(0,domain(x) - reps(x.g*w/2,x.g*w/2,x.d2,x.g))
    collisions_when_R_good = domain(x) - 2*reps(x.g*w/4+x.d2,x.g*w/4+x.d2,x.d1,x.g)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision

domain = lambda x : multiH(x.g,[(x.g*w/4+x.d2)*.5+x.d1,(x.g*w/4+x.d2)*.5+x.d1]) + multiH((1-x.g)*.25,[(1-x.g)*w*.125,(1-x.g)*w*.125])

constraints_lwe = [
# domain muss be equal to 1/2
{ 'type' : 'eq',   'fun' : lwe(lambda x : multiH(1,[w/2,w/2])*.5-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps(x.g*w/2,x.g*w/2,x.d2,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps(x.g*w/4+x.d2,x.g*w/4+x.d2,x.d1,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.g*w-2*x.d2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.g*w/2-2*x.d2-2*x.d1)}

]


bounds=[(0,0.1)]*2+[(0.01,1)]
for i in range(25,26):
    w = i*.02
    res = 100
    for j in range(4):
        start = [uniform(0,.009), uniform(0,.009), uniform(0.1,1)]  
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4))) 

(0.5, 0.8529)


For Nested-1*

In [11]:
def reps(p, m, d, l): 
    """
    representations of length-l vector with p ones and m minus ones
    two length-l vectors with p/2+d ones and d minus ones each.
    """
    if p <= 0.000001 or l == 0.:
        return 0
    if l < p or l - p < 2*d:
        return 0.
    
    return p + m + multiH(l-p-m, [d,d]) 


set_vars = collections.namedtuple('LWE', 'd1 d2 g b')
def lwe(f) : return wrap(f, set_vars)


p2 = lambda x : x.b*x.g*w/4+x.d2
p22 = lambda x : x.b*x.g*w/4+x.d2  #essentially same as p2


p1 = lambda x : p2(x)/2+x.d1
p11 = lambda x : p2(x)/2+x.d1      #essentially same as p1


def time(x):
    x=set_vars(*x)
    good_D = multiH(1,[w/2,w/2]) - ss(x)
    good_R = max(0,domain(x) - reps(x.b*x.g*w/2,x.b*x.g*w/2,x.d2,x.g))
    collisions_when_R_good = domain(x) - 2*reps(p2(x),p22(x),x.d1,x.g)
    one_collision=domain(x)
    return good_R+collisions_when_R_good+one_collision+good_D



domain = lambda x : multiH(x.g,[p1(x),p11(x)]) + multiH((1-x.g)*.25,[(1-x.b*x.g)*w*.125,(1-x.b*x.g)*w*.125])
ss = lambda x : multiH(x.g,[x.g*x.b*w/2, x.g*x.b*w/2]) + 4*multiH((1-x.g)/4,[(1-x.g*x.b)*w*.125,(1-x.g*x.b)*w*.125])

constraints_lwe = [
# domain must be equal to 1/2
{ 'type' : 'eq',   'fun' : lwe(lambda x : ss(x)*.5-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - reps(x.b*x.g*w/2,x.b*x.g*w/2,x.d2,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : domain(x) - 2*reps(p2(x),p22(x),x.d1,x.g))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.b - (1/x.g)+(1/(x.g*w))-(1/w))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-x.b)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x :  multiH(1,[w/2,w/2]) - ss(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.b*x.g*w-2*x.d2)},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : x.g-x.b*x.g*w/2-2*x.d2-2*x.d1)}

]


bounds=[(0,0.1)]*2+[(0.01,1)]*2
res = 100
for i in range(25, 26):
    w = i*.02
    res = 100
    for j in range(5):
        start = [uniform(0,.009), uniform(0,.009), uniform(0.1,1), uniform(0.1,1)]        
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        #print(result)
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4)))
    

(0.5, 0.8505)


For van Vredendaal's instantiation.

In [12]:
def time(w):
    tvv = multiH(1/2,[w/4,w/4])
    tvv = tvv*(3/2)
    return tvv

for i in range(25,26):
    w = .02*i
    print((round(w,2) , round(time(w),4)))

(0.5, 1.125)


For May's Rep-0 instantiation.

In [13]:
def time(w):
    T1 = multiH(1,[w/4,w/4])
    R = 2*binomH(w/2,w/4)
    Trep0 = T1*(3/2)-R
    return Trep0

for i in range(25,26):
    w = .02*i
    print((round(w,2) , round(time(w),4)))

(0.5, 1.0919)


For May's Rep-1 instantiation.

In [14]:
set_vars = collections.namedtuple('LWE', 'd')
def lwe(f) : return wrap(f, set_vars)

def time(x):
    x=set_vars(*x)
    T1 = multiH(1,[w/4+x.d,w/4+x.d])
    R = 2*binomH(w/2,w/4) + multiH(1-w,[x.d,x.d])
    Trep1 = 1.5*T1 - R
    return Trep1

domain = lambda x : multiH(1,[w/4+x.d,w/4+x.d])
ss = lambda x : multiH(1,[w/2,w/2])
constraints_lwe = [
#{ 'type' : 'ineq',   'fun' : lwe(lambda x : ss(x)-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -2*x.d+1-w)},
]

start=[.001]
bounds=[(0,0.1)]
for i in range(25,26):
    w = i*.02
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-12, 
            constraints=constraints_lwe, options={'maxiter':2000})
    r = result.get('fun')
    print((round(w,4),round(r,4)))

(0.5, 0.9635)


For May's Rep-2 instantiation.

In [15]:
set_vars = collections.namedtuple('LWE', 'e21 e20 e10')
def lwe(f) : return wrap(f, set_vars)

n2 = lambda x :x.e21+x.e20
n1 = lambda x :w/4 + x.e10
n0 = lambda x :1-w/2-2*(x.e10+x.e21+x.e20)


def time(x):
    x=set_vars(*x)
    T1 = multiH(1,[n2(x),n2(x),n1(x),n1(x)])
    R = 2*multiH(w/2,[x.e21,x.e21,w/4-x.e21]) + multiH(1-w,[x.e20,x.e20,x.e10,x.e10])
    Trep1 = 1.5*T1 - R
    return Trep1


domain = lambda x : multiH(1,[x.e21+x.e20,x.e21+x.e20,w/4 + x.e10,w/4 + x.e10])
ss = lambda x : multiH(1,[w/2,w/2])
constraints_lwe = [
#{ 'type' : 'ineq',   'fun' : lwe(lambda x : ss(x)-domain(x))},
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e21+w/4)},   
{ 'type' : 'ineq',   'fun' : lwe(lambda x : -x.e10-x.e20+1/2-w/2)},   

]


res = 100
bounds=[(0,0.1)]*3
for i in range(25,26):
    w = i*.02
    res = 100
    for j in range(10):
        start = [uniform(0,.009), uniform(0,.009), uniform(0,.009)]
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
            res = r
    print((round(w,4),round(res,4)))
    
    
    

(0.5, 0.962)
